In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentiment-analysis-for-financial-news/all-data.csv
/kaggle/input/sentiment-analysis-for-financial-news/FinancialPhraseBank/Sentences_66Agree.txt
/kaggle/input/sentiment-analysis-for-financial-news/FinancialPhraseBank/Sentences_AllAgree.txt
/kaggle/input/sentiment-analysis-for-financial-news/FinancialPhraseBank/README.txt
/kaggle/input/sentiment-analysis-for-financial-news/FinancialPhraseBank/License.txt
/kaggle/input/sentiment-analysis-for-financial-news/FinancialPhraseBank/Sentences_75Agree.txt
/kaggle/input/sentiment-analysis-for-financial-news/FinancialPhraseBank/Sentences_50Agree.txt


# Step 1 - Read in and prepare our data

In [2]:
df = pd.read_csv('/kaggle/input/sentiment-analysis-for-financial-news/all-data.csv',encoding='latin1'
                ,header=None,names=['rating','text'])
df.head()

,rating,text
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...


In [3]:
from keras.preprocessing.text import Tokenizer

tokenizer=Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df.text.values)

tokenized = tokenizer.texts_to_sequences(df.text.values)
sen_length = df.text.apply(lambda x: len(x))
sen_length.value_counts()

102    52
95     49
96     46
67     43
91     43
       ..
17      1
315     1
305     1
9       1
263     1
Name: text, Length: 285, dtype: int64

In [4]:
from keras.preprocessing.sequence import pad_sequences

X = pad_sequences(tokenized,maxlen=200)
X

array([[  0,   0,   0, ...,  11,  10, 747],
       [  0,   0,   0, ...,   1, 929,  19],
       [  0,   0,   0, ...,   1, 877, 251],
       ...,
       [  0,   0,   0, ..., 129,  38,  23],
       [  0,   0,   0, ...,  57,  54,  23],
       [  0,   0,   0, ..., 962,  18, 295]], dtype=int32)

In [5]:
df['rating'] = df['rating'].replace('neutral',0)
df['rating'] = df['rating'].replace('positive',1)
df['rating'] = df['rating'].replace('negative',2)
df

,rating,text
0,0,"According to Gran , the company has no plans t..."
1,0,Technopolis plans to develop in stages an area...
2,2,The international electronic industry company ...
3,1,With the new production plant the company woul...
4,1,According to the company 's updated strategy f...
...,...,...
4841,2,LONDON MarketWatch -- Share prices ended lower...
4842,0,Rinkuskiai 's beer sales fell by 6.5 per cent ...
4843,2,Operating profit fell to EUR 35.4 mn from EUR ...
4844,2,Net sales of the Paper segment decreased to EU...


In [6]:
from keras.utils.np_utils import to_categorical

y = to_categorical(df['rating'])
y

array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [7]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.3,random_state=42)

# Step 2 - Create and train our model!

In [8]:
from keras.models import Sequential
from keras.layers import Dense,SimpleRNN,GRU,LSTM,Embedding

model = Sequential()

model.add(Embedding(5000,64))
model.add(SimpleRNN(64,return_sequences=True))
model.add(SimpleRNN(64))
model.add(Dense(3,activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

2022-03-16 01:57:26.088547: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          320000    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, None, 64)          8256      
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 64)                8256      
_________________________________________________________________
dense (Dense)                (None, 3)                 195       
Total params: 336,707
Trainable params: 336,707
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.fit(X_train,y_train,epochs=5,batch_size=10)

Epoch 1/5


2022-03-16 01:57:26.453351: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


340/340 [==============================] - 38s 105ms/step - loss: 0.8669 - accuracy: 0.6268
Epoch 2/5
340/340 [==============================] - 35s 103ms/step - loss: 0.6469 - accuracy: 0.7273
Epoch 3/5
340/340 [==============================] - 35s 104ms/step - loss: 0.4364 - accuracy: 0.8264
Epoch 4/5
340/340 [==============================] - 36s 105ms/step - loss: 0.5524 - accuracy: 0.7686
Epoch 5/5
340/340 [==============================] - 36s 105ms/step - loss: 0.5128 - accuracy: 0.7804
